Import data from datasheet for ZTP-135SR in regards to the thermistor resistance values as a function of temperature

In [1]:
import pandas as pd
import numpy as np
import array as array
import os
import ipywidgets as widgets
import matplotlib.pyplot as plt

%matplotlib widget

In [2]:
thermistor_res_directory = "calibration_data/thermistor_res.csv"
thermistor_res = pd.read_csv(thermistor_res_directory)

# change celsius to kelvin
thermistor_res['Tamb'] = thermistor_res['Tamb'] + 273.15

plot thermistor resistance vs temperature

In [3]:
thermistor_res.plot(x='Tamb', y=["Rmin","Rcent","Rmax"])
plt.title("Resistance vs. Temperature")
plt.ylabel("Resistance (kOhms)")
plt.xlabel("Temperature (Kelvin)")
plt.show()
# plt.savefig("calibration_data/plots/thermistor_temp_curve.png")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

find best fit curve

In [4]:
# curve-fit() function imported from scipy 
from scipy.optimize import curve_fit 
import math as math

In [5]:
# Test function with coefficients as parameters 
# def fit_func(x, a, b, c): 
#     return a * pow(b,x) + c

# Steinhart-Hart Equation
# ref: https://www.northstarsensors.com/calculating-temperature-from-resistance
# Steinhart-Hart-Equation
def res_to_temp(x, a0, a1, a2, a3):
    return 1. / (a0 + a1 * np.log(x) + a2 * np.power(np.log(x),2) + a3 * np.power(np.log(x),3))

# curve_fit() function takes the test-function 
# x-data and y-data as argument and returns  
# the coefficients a and b in param and 
# the estimated covariance of param in param_cov 
x = thermistor_res['Rcent'].values
y = thermistor_res['Tamb'].values
# Do the fit with initial values (needed, otherwise no meaningful result)
param_thermistor, param_cov_thermistor = curve_fit(res_to_temp, x, y, p0=[1e-4, 1e-4, 1e-4, 1e-4])

print("Function coefficients:") 
print(param_thermistor) 
print("Covariance of coefficients:") 
print(param_cov_thermistor) 

# generate synthetic data
x_synthetic = np.linspace(5,1000,num=300)
synthetic_output = res_to_temp(x_synthetic, param_thermistor[0], param_thermistor[1], param_thermistor[2], param_thermistor[3])

# plt.figure()
ax = thermistor_res.plot(x='Tamb', y=["Rmin","Rcent","Rmax"])
plt.sca(ax)
plt.plot(synthetic_output, x_synthetic, label="Best Fit")

plt.title("Resistance vs. Temperature")
plt.ylabel("Resistance (kOhms)")
plt.xlabel("Temperature (Kelvin)")

plt.legend()
plt.show()

Function coefficients:
[ 2.26493387e-03  2.14597062e-04  5.13860990e-06 -8.41555239e-08]
Covariance of coefficients:
[[ 1.52654574e-13 -1.26315940e-13  3.17434564e-14 -2.46499993e-15]
 [-1.26315940e-13  1.06219994e-13 -2.70572380e-14  2.12393984e-15]
 [ 3.17434564e-14 -2.70572380e-14  6.98621779e-15 -5.54865805e-16]
 [-2.46499993e-15  2.12393984e-15 -5.54865805e-16  4.45537118e-17]]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Lets bring in the calibration data that was collected

In [6]:
thermistor_pullup = 100 #kOhms
thermopile_gain = 1344
thermopile_offset = 1.7584

nose_thermopile_cal = pd.read_csv("calibration_data/nose_thermopile_cal.csv")
temple_thermopile_cal = pd.read_csv("calibration_data/temple_thermopile_cal.csv")

Time to apply some preprocessing conversions

In [7]:
#convert thermopile ADC values to voltage (taking into consideration offset offered by analog front end IC)
nose_thermopile_cal['thermopile voltage'] = 3.3*(nose_thermopile_cal['thermopile']/4096.) - thermopile_offset
temple_thermopile_cal['thermopile voltage'] = 3.3*(temple_thermopile_cal['thermopile']/4096.) - thermopile_offset

#convert thermistor ADC values to thermistor resistance, knowing that thermistor is pulled up by 100k resistor
nose_thermopile_cal['thermistor resistance'] = ((nose_thermopile_cal['thermistor']/4096.) * thermistor_pullup) / ( 1 - (nose_thermopile_cal['thermistor']/4096.))
temple_thermopile_cal['thermistor resistance'] = ((temple_thermopile_cal['thermistor']/4096.) * thermistor_pullup) / ( 1 - (temple_thermopile_cal['thermistor']/4096.))

#convert resistances to temperature
nose_thermopile_cal['thermistor temp'] = res_to_temp(nose_thermopile_cal['thermistor resistance'], param_thermistor[0], param_thermistor[1], param_thermistor[2], param_thermistor[3])
temple_thermopile_cal['thermistor temp'] = res_to_temp(temple_thermopile_cal['thermistor resistance'], param_thermistor[0], param_thermistor[1], param_thermistor[2], param_thermistor[3])

#convert peltier temperature to kelvin
nose_thermopile_cal['thermocouple peltier kelvin'] = (nose_thermopile_cal['thermocouple_peltier'] + 459.67) * 5./9.
temple_thermopile_cal['thermocouple peltier kelvin'] = (temple_thermopile_cal['thermocouple_peltier'] + 459.67) * 5./9.

Lets define the thermopile equations

In [8]:
# https://www.heimannsensor.com/Application%20Note%20Thermopile%20Module%20Analog.pdf
# https://www.fierceelectronics.com/components/demystifying-thermopile-ir-temp-sensors
# https://media.digikey.com/pdf/Data%20Sheets/Texas%20Instruments%20PDFs/TMP006(B).pdf
#    note: assumption is we are neglecting emmisivity .... this can be a false assumption depending on skin color
def thermopile_voltage_equation(X, a0):
    temp_obj, temp_sensor = X
    return a0 * (pow(temp_obj,4) - pow(temp_sensor,4)) 

# def temperature_of_object(X, a0):
#     thermopile_voltage, temp_sensor = X
#     return pow((thermopile_voltage/a0) + pow(temp_sensor,4), 1/4.)

def voltage_correction(thermopile_voltage, a1, a2):
    return (thermopile_voltage - a1) + a2*pow((thermopile_voltage - a1),2)

def temperature_of_object(X, a0, a1, a2):
    thermopile_voltage, temp_sensor = X

    base = ((voltage_correction(thermopile_voltage,a1,a2))/a0) + pow(temp_sensor,4)
    
    # if less than negative
    if isinstance(base, np.ndarray):
        base[base < 0] = 0
    else:
        if base < 0:
            base = 0
    
    return pow( base, 1/4.)

Need to solve for coefficients

In [9]:
# SOLVE FOR NOSE
y = nose_thermopile_cal['thermocouple peltier kelvin'].values
x_1 = nose_thermopile_cal['thermopile voltage'].values
x_2 = nose_thermopile_cal['thermistor temp'].values

a_0_estimate = 0.95 * 5.7 * pow(10,-12) # emissivity * Boltzman constant
a_1_estimate = 0
a_2_estimate = -10
# Do the fit with initial values (needed, otherwise no meaningful result)
param_nose_thermopile, param_cov_nose_thermopile = curve_fit(temperature_of_object, (x_1,x_2), y, p0=[a_0_estimate, a_1_estimate, a_2_estimate])

print("NOSE") 
print("Function coefficients:") 
print(param_nose_thermopile) 
print("Covariance of coefficients:") 
print(param_cov_nose_thermopile) 


print("")
print("experimental values")
print(y)
print("fit values")
print(temperature_of_object((x_1,x_2),param_nose_thermopile[0],param_nose_thermopile[1],param_nose_thermopile[2]))

print("") 
########################################################################################################################

# SOLVE FOR TEMPLE
y = temple_thermopile_cal['thermocouple peltier kelvin'].values
x_1 = temple_thermopile_cal['thermopile voltage'].values
x_2 = temple_thermopile_cal['thermistor temp'].values

a_0_estimate = 0.95 * 5.7 * pow(10,-12) # emissivity * Boltzman constant
a_1_estimate = 0
a_2_estimate = -10
# Do the fit with initial values (needed, otherwise no meaningful result)
param_temple_thermopile, param_cov_temple_thermopile = curve_fit(temperature_of_object, (x_1,x_2), y, p0=[a_0_estimate, a_1_estimate, a_2_estimate])

print("TEMPLE") 
print("Function coefficients:") 
print(param_temple_thermopile) 
print("Covariance of coefficients:") 
print(param_cov_temple_thermopile) 


print("")
print("experimental values")
print(y)
print("fit values")
print(temperature_of_object((x_1,x_2),param_temple_thermopile[0],param_temple_thermopile[1],param_temple_thermopile[2]))

NOSE
Function coefficients:
[ 7.80334836e-10 -2.30649152e-01  3.61481433e-03]
Covariance of coefficients:
[[1.60558806e-23 8.83743078e-15 1.72207609e-14]
 [8.83743078e-15 1.82663690e-05 1.90409273e-05]
 [1.72207609e-14 1.90409273e-05 3.36518437e-05]]

experimental values
[317.81666667 313.65       307.70555556 304.98333333 302.53888889
 311.26111111 295.15       293.31666667 292.31666667 290.98333333
 290.48333333]
fit values
[317.73098386 313.71274318 307.83007091 304.98249695 302.50763442
 311.26308595 294.93625132 293.33860667 292.27012828 291.06924551
 290.56442721]

TEMPLE
Function coefficients:
[ 4.21293053e-10 -3.61646238e-01  8.31103607e-02]
Covariance of coefficients:
[[4.22092531e-23 8.98459323e-15 1.57210701e-13]
 [8.98459323e-15 1.78470167e-05 7.36605730e-05]
 [1.57210701e-13 7.36605730e-05 8.41121281e-04]]

experimental values
[316.87222222 313.37222222 309.48333333 306.59444444 303.59444444
 300.59444444 296.53888889 294.20555556 293.15       292.09444444
 290.03888889]
f

Save parameters

In [10]:
np.save("calibration_data/param_nose_thermopile", param_nose_thermopile)
np.save("calibration_data/param_temple_thermopile", param_temple_thermopile)